<a href="https://colab.research.google.com/github/Mayhemy/Smith-Waterman-Using-PyCuda/blob/main/P3_Paralelni_algoritmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp38-cp38-linux_x86_64.whl size=646530 sha256=af1fbfd7f7f36353c2c053fe76a59a640a8757ad8ff1bd41f22b603ec0511ba5
  Stored in directory: /root/.cache/pip/wheels/7b/41/0d/7cecb04af969d283ebe4a69579a8b2baec0d010a1ac4159f7e
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69870 sha256=bd654402f1a5e3b6501e31e108efbf5d2d5d2f8587977601c2bf69bf19ca8ec2
  Stored in directory: /root/.cache/pip/wheels/cb/fc/

1. CUDA paralelna implementacija izačuavanja matrice

In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import math

# define CUDA kernel function for diagonal filling
mod = SourceModule("""
    __global__ void diagonal_fill(int *matrix, int diagonal_len, int max_diagonal_len, int *seq1, int *seq2, int n, int m, int gap_penalty) {
        int idx = threadIdx.x;
        int i = (diagonal_len <= max_diagonal_len) ? idx : diagonal_len % max_diagonal_len + idx;
        int j = (diagonal_len <= max_diagonal_len) ? diagonal_len - idx : diagonal_len - (diagonal_len % max_diagonal_len) - idx;
        if ((idx < diagonal_len) && i < n && j < m) {
            int left_score = (j > 0) ? matrix[i*m + j-1] + gap_penalty : 0;
            int up_score = (i > 0) ? matrix[(i-1)*m + j] + gap_penalty : 0;
            int match_score = (i > 0 && j > 0) ? matrix[(i-1)*m + j-1] + (seq1[j] == seq2[i] ? 5 : -3) : 0;
            matrix[i*m + j] = max(0, max(left_score, max(up_score, match_score)));
        }
    }
""")

def SmithWaterman(seq1, seq2, gap_penalty):
    n = len(seq1)
    m = len(seq2)
    max_diag_len = min(n, m)

    # create CUDA arrays for the sequences and the alignment matrix
    seq1_gpu = cuda.mem_alloc(seq1.nbytes)
    seq2_gpu = cuda.mem_alloc(seq2.nbytes)
    matrix_gpu = cuda.mem_alloc(n*m*4)

    # copy sequences and matrix to the GPU
    cuda.memcpy_htod(seq1_gpu, seq1)
    cuda.memcpy_htod(seq2_gpu, seq2)
    cuda.memcpy_htod(matrix_gpu, np.zeros((n, m), dtype=np.int32))

    # compile the CUDA kernel
    diagonal_fill = mod.get_function("diagonal_fill")

    # call the kernel for each diagonal
    block_size = (max_diag_len, 1, 1)
    grid_size = (1, 1, 1)
    sum = 0
    for i in range(n+m):
      # for j in range(i+1):
      #   indexes.append(np.array([i-j,j], dtype=np.int32))
      # indexes = np.array(indexes)
      # indexes_gpu = cuda.mem_alloc(indexes.nbytes)
      # cuda.memcpy_htod(indexes_gpu, indexes)
      duration = diagonal_fill(matrix_gpu, np.int32(i), np.int32(max_diag_len), seq1_gpu, seq2_gpu, np.int32(n), np.int32(m), np.int32(gap_penalty), block=block_size, grid=grid_size, time_kernel=True)
      sum += duration

    print(sum, 'ms')
    # copy matrix back from GPU to host
    matrix = np.empty((n, m), dtype=np.int32)
    cuda.memcpy_dtoh(matrix, matrix_gpu)
    return matrix

def create_alignment_matrix(seq1, seq2, match_score=5, mismatch_score=-3, gap_score=-9):
    rows = len(seq1)
    cols = len(seq2)
    # Create the alignment matrix filled with 0
    alignment_matrix = np.zeros((rows,cols), dtype = int)

    # fill the matrix with scores
    for i in range(1, rows):
        for j in range(1, cols):
            # calculate the scores based on the surrounding cells
            to_add = 0
            if i <= cols and j < rows :
              if seq1[j] == seq2[i]:
                to_add = 5
              else:
                to_add = -3
              diagonal_score = alignment_matrix[i-1][j-1] + to_add
              up_score = alignment_matrix[i-1][j] + gap_score
              left_score = alignment_matrix[i][j-1] + gap_score
              alignment_matrix[i][j] = max(0, diagonal_score, up_score, left_score)
    return alignment_matrix

seq1 = '0CAGCCUCGCUUAG'
seq2 = '0AAUGCCAUUGCCGG'
gap_penalty = -9
print(np.array(list(seq1)))
print(np.array(list(seq2)))
matrix = SmithWaterman(np.array(list(seq1)),np.array(list(seq2)), gap_penalty)
matrix1 = create_alignment_matrix(np.array(list(seq1)),np.array(list(seq2)), gap_penalty)
# np.insert(matrix1, 0, int(seq1),axis = 0)
# np.insert(matrix1, 1, int(seq2),axis = 1)
print(matrix)
print()
print(matrix1)

['0' 'C' 'A' 'G' 'C' 'C' 'U' 'C' 'G' 'C' 'U' 'U' 'A' 'G']
['0' 'A' 'A' 'U' 'G' 'C' 'C' 'A' 'U' 'U' 'G' 'C' 'C' 'G' 'G']
0.00039386749267578125 ms
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2  0]
 [ 0  0  0  2  0  0  5  0  0  0  5  5  0  2  0]
 [ 0  0  0  5  0  0  0  2  5  0  0  2  2  5  0]
 [ 0  5  0  0 10  5  0  5  0 10  1  0  0  0  2]
 [ 0  5  2  0  5 15  6  5  2  5  7  0  0  0  0]
 [ 0  0 10  1  0  6 12  3  2  0  2  4  5  0  0]
 [ 0  0  1  7  0  0 11  9  0  0  5  7  1  2  0]
 [ 0  0  0  0  4  0  5  8  6  0  5 10  4  0  0]
 [ 0  0  0  5  0  1  0  2 13  4  0  2  7  9  0]
 [ 0  5  0  0 10  5  0  5  4 18  9  0  0  4  6]
 [ 0  5  2  0  5 15  6  5  2  9 15  6  0  0  1]
 [ 0  0  2  7  0  6 12  3 10  1  6 12  3  5  0]]

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2  0]
 [ 0  0  0  2  0  0  5  0  0  0  5  

2. CUDA paralelna implementacija pronalaženja optimalne vrednosti poravanja

In [ ]:
def cuda_alloc(*args):
  return [cuda.mem_alloc(arg.nbytes) for arg in args]


def to_cuda(*args):
  cuda_ptrs = cuda_alloc(*args)
  for dst, src in zip(cuda_ptrs, args):
    cuda.memcpy_htod(dst, src)
  return cuda_ptrs

mod = SourceModule("""
    __global__ void reduce(int *a, float *result, int width){

      int idx = threadIdx.x;

      for(unsigned int s=1; s < blockDim.x; s *= 2) {

         if (idx % (2*s) == 0){
            a[idx] = max(a[idx], a[idx + s]);
          }
          __syncthreads();
      }

      // write result for this block to global mem
      if (idx == 0){
        result[0] = a[0];
      }

    }
  """)


reduction_kernel = mod.get_function("reduce")

matrix_gpu = cuda.mem_alloc(matrix.shape[0] * matrix.shape[1] *4)
# cuda.memcpy_htod(matrix_gpu, matrix)

matrix_gpu = gpuarray.to_gpu(matrix)
# matrix_gpu = cuda_alloc(matrix)
# to_cuda(matrix_gpu)
print(matrix)
# cuda.memcpy_htod(matrix_gpu, np.zeros((matrix.shape[0], matrix.shape[1]), dtype=np.int32))

result = np.zeros(1, dtype=np.float32)
result_cuda = to_cuda(result)[0]

reduction_kernel(matrix_gpu, result_cuda, np.int32(matrix.shape[1]),block=(matrix.shape[0] * matrix.shape[1], 1, 1), grid=(1, 1, 1))

cuda.memcpy_dtoh(result, result_cuda)

print(np.round(result,1))

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2  0]
 [ 0  0  0  2  0  0  5  0  0  0  5  5  0  2  0]
 [ 0  0  0  5  0  0  0  2  5  0  0  2  2  5  0]
 [ 0  5  0  0 10  5  0  5  0 10  1  0  0  0  2]
 [ 0  5  2  0  5 15  6  5  2  5  7  0  0  0  0]
 [ 0  0 10  1  0  6 12  3  2  0  2  4  5  0  0]
 [ 0  0  1  7  0  0 11  9  0  0  5  7  1  2  0]
 [ 0  0  0  0  4  0  5  8  6  0  5 10  4  0  0]
 [ 0  0  0  5  0  1  0  2 13  4  0  2  7  9  0]
 [ 0  5  0  0 10  5  0  5  4 18  9  0  0  4  6]
 [ 0  5  2  0  5 15  6  5  2  9 15  6  0  0  1]
 [ 0  0  2  7  0  6 12  3 10  1  6 12  3  5  0]]
[18.]


In [ ]:
def cuda_alloc(*args):
  return [cuda.mem_alloc(arg.nbytes) for arg in args]


def to_cuda(*args):
  cuda_ptrs = cuda_alloc(*args)
  for dst, src in zip(cuda_ptrs, args):
    cuda.memcpy_htod(dst, src)
  return cuda_ptrs

mod = SourceModule("""
    __global__ void reduce(int *a, float *result, int width){

      int idx = threadIdx.x + threadIdx.y*width ;

      for(unsigned int s=1; s < blockDim.x + blockDim.y*width; s *= 2) {

         if (idx % (2*s) == 0){
            a[idx] = max(a[idx], a[idx + s]);
          }
          __syncthreads();
      }

      // write result for this block to global mem
      if (idx == 1){
        result[0] = a[0];
      }

    }
  """)


reduction_kernel = mod.get_function("reduce")

matrix_gpu = cuda.mem_alloc(matrix.shape[0] * matrix.shape[1] *4)
# cuda.memcpy_htod(matrix_gpu, matrix)

matrix_gpu = gpuarray.to_gpu(matrix)
# matrix_gpu = cuda_alloc(matrix)
# to_cuda(matrix_gpu)
print(matrix)
# cuda.memcpy_htod(matrix_gpu, np.zeros((matrix.shape[0], matrix.shape[1]), dtype=np.int32))

result = np.zeros(1, dtype=np.float32)
result_cuda = to_cuda(result)[0]

reduction_kernel(matrix_gpu, result_cuda, np.int32(matrix.shape[1]),block=(matrix.shape[1], matrix.shape[0], 1), grid=(1, 1, 1))

cuda.memcpy_dtoh(result, result_cuda)

print(np.round(result,1))

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2  0]
 [ 0  0  0  2  0  0  5  0  0  0  5  5  0  2  0]
 [ 0  0  0  5  0  0  0  2  5  0  0  2  2  5  0]
 [ 0  5  0  0 10  5  0  5  0 10  1  0  0  0  2]
 [ 0  5  2  0  5 15  6  5  2  5  7  0  0  0  0]
 [ 0  0 10  1  0  6 12  3  2  0  2  4  5  0  0]
 [ 0  0  1  7  0  0 11  9  0  0  5  7  1  2  0]
 [ 0  0  0  0  4  0  5  8  6  0  5 10  4  0  0]
 [ 0  0  0  5  0  1  0  2 13  4  0  2  7  9  0]
 [ 0  5  0  0 10  5  0  5  4 18  9  0  0  4  6]
 [ 0  5  2  0  5 15  6  5  2  9 15  6  0  0  1]
 [ 0  0  2  7  0  6 12  3 10  1  6 12  3  5  0]]
[18.]


3. Upotreba deljene i konstante memorije
a) Konstantna memorija


In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray

# define CUDA kernel function for diagonal filling
mod = SourceModule("""
      __constant__  float seq1[14];
      __constant__  float seq2[15];


    __global__ void diagonal_fill_adv(int *matrix, int diagonal_len, int max_diagonal_len, int n, int m, int gap_penalty) {

        int idx = threadIdx.x;

        int i = (diagonal_len <= max_diagonal_len) ? idx : diagonal_len % max_diagonal_len + idx;
        int j = (diagonal_len <= max_diagonal_len) ? diagonal_len - idx : diagonal_len - (diagonal_len % max_diagonal_len) - idx;
        if ((idx < diagonal_len) && i < n && j < m) {
            int left_score = (j > 0) ? matrix[i*m + j-1] + gap_penalty : 0;
            int up_score = (i > 0) ? matrix[(i-1)*m + j] + gap_penalty : 0;
            int match_score = (i > 0 && j > 0) ? matrix[(i-1)*m + j-1] + (seq1[j] == seq2[i] ? 5 : -3) : 0;
            matrix[i*m + j] = max(0, max(left_score, max(up_score, match_score)));
        }
    }
""")

def SmithWatermanAdv(seq1, seq2, gap_penalty):
    n = len(seq1)
    m = len(seq2)
    max_diag_len = min(n, m)

    # create CUDA arrays for the sequences and the alignment matrix

    matrix_gpu = cuda.mem_alloc(n*m*4)

    # za shared
    # seq1_cuda = cuda.mem_alloc(seq1.nbytes)
    # seq2_cuda = cuda.mem_alloc(seq2.nbytes)
    # za const zakomentarisana prosla dva reda i otkomentarisana sledeca dva
    seq1_cuda = mod.get_global('seq1')[0]
    seq2_cuda = mod.get_global('seq2')[0]

    cuda.memcpy_htod(seq1_cuda, seq1)
    cuda.memcpy_htod(seq2_cuda, seq2)


    # copy sequences and matrix to the GPU
    cuda.memcpy_htod(matrix_gpu, np.zeros((n, m), dtype=np.int32))

    # compile the CUDA kernel
    diagonal_fill = mod.get_function("diagonal_fill_adv")

    # call the kernel for each diagonal
    block_size = (max_diag_len, 1, 1)
    grid_size = (1, 1, 1)
    sum = 0
    for i in range(n+m):
      # for j in range(i+1):
      #   indexes.append(np.array([i-j,j], dtype=np.int32))
      # indexes = np.array(indexes)
      # indexes_gpu = cuda.mem_alloc(indexes.nbytes)
      # cuda.memcpy_htod(indexes_gpu, indexes)
      duration = diagonal_fill(matrix_gpu, np.int32(i), np.int32(max_diag_len), np.int32(n), np.int32(m), np.int32(gap_penalty), block=block_size, grid=grid_size, time_kernel=True)
      sum += duration

    print(sum, 'ms')

    # copy matrix back from GPU to host
    matrix = np.empty((n, m), dtype=np.int32)
    cuda.memcpy_dtoh(matrix, matrix_gpu)
    return matrix

seq1 = '0CAGCCUCGCUUAG'
seq2 = '0AAUGCCAUUGCCGG'
gap_penalty = -9
print(np.array(list(seq1)))
print(np.array(list(seq2)))
matrix = SmithWatermanAdv(np.array(list(seq1)),np.array(list(seq2)), gap_penalty)
matrix1 = create_alignment_matrix(np.array(list(seq1)),np.array(list(seq2)), gap_penalty)
# np.insert(matrix1, 0, int(seq1),axis = 0)
# np.insert(matrix1, 1, int(seq2),axis = 1)
print(matrix)
print()
print(matrix1)

['0' 'C' 'A' 'G' 'C' 'C' 'U' 'C' 'G' 'C' 'U' 'U' 'A' 'G']
['0' 'A' 'A' 'U' 'G' 'C' 'C' 'A' 'U' 'U' 'G' 'C' 'C' 'G' 'G']
0.00042891502380371094 ms
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2  0]
 [ 0  0  0  2  0  0  5  0  0  0  5  5  0  2  0]
 [ 0  0  0  5  0  0  0  2  5  0  0  2  2  5  0]
 [ 0  5  0  0 10  5  0  5  0 10  1  0  0  0  2]
 [ 0  5  2  0  5 15  6  5  2  5  7  0  0  0  0]
 [ 0  0 10  1  0  6 12  3  2  0  2  4  5  0  0]
 [ 0  0  1  7  0  0 11  9  0  0  5  7  1  2  0]
 [ 0  0  0  0  4  0  5  8  6  0  5 10  4  0  0]
 [ 0  0  0  5  0  1  0  2 13  4  0  2  7  9  0]
 [ 0  5  0  0 10  5  0  5  4 18  9  0  0  4  6]
 [ 0  5  2  0  5 15  6  5  2  9 15  6  0  0  1]
 [ 0  0  2  7  0  6 12  3 10  1  6 12  3  5  0]]

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2  0]
 [ 0  0  0  2  0  0  5  0  0  0  5  

b) Deljena memorija

In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray

# define CUDA kernel function for diagonal filling
mod = SourceModule("""
    __global__ void diagonal_fill(int *matrix, int diagonal_len, int max_diagonal_len, int *seq1, int *seq2, int n, int m, int gap_penalty) {

          __shared__ int seq1_shared[14*4];
          __shared__ int seq2_shared[14*4];

          int idx = threadIdx.x;

          if (threadIdx.x < n) {
              seq1_shared[threadIdx.x] = seq1[threadIdx.x];
          }
          __syncthreads();

          if (threadIdx.x < m) {
              seq2_shared[threadIdx.x] = seq2[threadIdx.x];
          }
          __syncthreads();

          int i = (diagonal_len <= max_diagonal_len) ? idx : diagonal_len % max_diagonal_len + idx;
          int j = (diagonal_len <= max_diagonal_len) ? diagonal_len - idx : diagonal_len - (diagonal_len % max_diagonal_len) - idx;
          if ((idx < diagonal_len) && i < n && j < m) {
              int left_score = (j > 0) ? matrix[i*m + j-1] + gap_penalty : 0;
              int up_score = (i > 0) ? matrix[(i-1)*m + j] + gap_penalty : 0;
              int match_score = (i > 0 && j > 0) ? matrix[(i-1)*m + j-1] + (seq1_shared[j] == seq2_shared[i] ? 5 : -3) : 0;
              matrix[i*m + j] = max(0, max(left_score, max(up_score, match_score)));
          }
    }
    """)

def SmithWaterman(seq1, seq2, gap_penalty):
    n = len(seq1)
    m = len(seq2)
    max_diag_len = min(n, m)

    # create CUDA arrays for the sequences and the alignment matrix
    seq1_gpu = cuda.mem_alloc(seq1.nbytes)
    seq2_gpu = cuda.mem_alloc(seq2.nbytes)
    matrix_gpu = cuda.mem_alloc(n*m*4)

    # copy sequences and matrix to the GPU
    cuda.memcpy_htod(seq1_gpu, seq1)
    cuda.memcpy_htod(seq2_gpu, seq2)
    cuda.memcpy_htod(matrix_gpu, np.zeros((n, m), dtype=np.int32))

    # compile the CUDA kernel
    diagonal_fill = mod.get_function("diagonal_fill")

    # call the kernel for each diagonal
    block_size = (max_diag_len, 1, 1)
    grid_size = (1, 1, 1)
    sum = 0
    for i in range(n+m):
      # for j in range(i+1):
      #   indexes.append(np.array([i-j,j], dtype=np.int32))
      # indexes = np.array(indexes)
      # indexes_gpu = cuda.mem_alloc(indexes.nbytes)
      # cuda.memcpy_htod(indexes_gpu, indexes)
      duration = diagonal_fill(matrix_gpu, np.int32(i), np.int32(max_diag_len), seq1_gpu, seq2_gpu, np.int32(n), np.int32(m), np.int32(gap_penalty), block=block_size, grid=grid_size, time_kernel=True)
      sum += duration

    print(sum, 'ms')
    # copy matrix back from GPU to host
    matrix = np.empty((n, m), dtype=np.int32)
    cuda.memcpy_dtoh(matrix, matrix_gpu)
    return matrix

def create_alignment_matrix(seq1, seq2, match_score=5, mismatch_score=-3, gap_score=-9):
    rows = len(seq1)
    cols = len(seq2)
    # Create the alignment matrix filled with 0
    alignment_matrix = np.zeros((rows,cols), dtype = int)

    # fill the matrix with scores
    for i in range(1, rows):
        for j in range(1, cols):
            # calculate the scores based on the surrounding cells
            to_add = 0
            if i <= cols and j < rows :
              if seq1[j] == seq2[i]:
                to_add = 5
              else:
                to_add = -3
              diagonal_score = alignment_matrix[i-1][j-1] + to_add
              up_score = alignment_matrix[i-1][j] + gap_score
              left_score = alignment_matrix[i][j-1] + gap_score
              alignment_matrix[i][j] = max(0, diagonal_score, up_score, left_score)
    return alignment_matrix

seq1 = '0CAGCCUCGCUUAG'
seq2 = '0AAUGCCAUUGCCGG'
gap_penalty = -9
print(np.array(list(seq1)))
print(np.array(list(seq2)))
matrix = SmithWaterman(np.array(list(seq1)),np.array(list(seq2)), gap_penalty)
matrix1 = create_alignment_matrix(np.array(list(seq1)),np.array(list(seq2)), gap_penalty)
# np.insert(matrix1, 0, int(seq1),axis = 0)
# np.insert(matrix1, 1, int(seq2),axis = 1)
print(matrix)
print()
print(matrix1)

['0' 'C' 'A' 'G' 'C' 'C' 'U' 'C' 'G' 'C' 'U' 'U' 'A' 'G']
['0' 'A' 'A' 'U' 'G' 'C' 'C' 'A' 'U' 'U' 'G' 'C' 'C' 'G' 'G']
0.0003788471221923828 ms
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0  5]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2  5]
 [ 0  0  0  2  0  0  5  0  0  0  5  5  0  2  0]
 [ 0  0  0  5  0  0  0  2  5  0  0  2  2  5  0]
 [ 0  5  0  0 10  5  0  5  0 10  1  0  0  0  2]
 [ 0  5  2  0  5 15  6  5  2  5  7  0  0  0  0]
 [ 0  0 10  1  0  6 12  3  2  0  2  4  5  0  5]
 [ 0  0  1  7  0  0 11  9  0  0  5  7  1  2  0]
 [ 0  0  0  0  4  0  5  8  6  0  5 10  4  0  0]
 [ 0  0  0  5  0  1  0  2 13  4  0  2  7  9  0]
 [ 0  5  0  0 10  5  0  5  4 18  9  0  0  4  6]
 [ 0  5  2  0  5 15  6  5  2  9 15  6  0  0  1]
 [ 0  0  2  7  0  6 12  3 10  1  6 12  3  5  0]]

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  5  2  0  0  0  0  0  0  0  0  5  2  0]
 [ 0  0  0  2  0  0  5  0  0  0  5  5

4. Podrška za dugačke sekvence

In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import math

# define CUDA kernel function for diagonal filling
mod = SourceModule("""
    __global__ void diagonal_fill(int *matrix, int diagonal_len, int max_diagonal_len, int *seq1, int *seq2, int n, int m, int gap_penalty) {
        //int idx = threadIdx.x;
        long idx = threadIdx.x + blockDim.x*blockIdx.x + threadIdx.y*width + blockDim.y*blockIdx.y*width;
        int i = (diagonal_len <= max_diagonal_len) ? idx : diagonal_len % max_diagonal_len + idx;
        int j = (diagonal_len <= max_diagonal_len) ? diagonal_len - idx : diagonal_len - (diagonal_len % max_diagonal_len) - idx;
        if ((idx < diagonal_len) && i < n && j < m) {
            int left_score = (j > 0) ? matrix[i*m + j-1] + gap_penalty : 0;
            int up_score = (i > 0) ? matrix[(i-1)*m + j] + gap_penalty : 0;
            int match_score = (i > 0 && j > 0) ? matrix[(i-1)*m + j-1] + (seq1[j] == seq2[i] ? 5 : -3) : 0;
            matrix[i*m + j] = max(0, max(left_score, max(up_score, match_score)));
        }
    }
""")

def SmithWaterman(seq1, seq2, gap_penalty):
    n = len(seq1)
    m = len(seq2)
    max_diag_len = min(n, m)

    # create CUDA arrays for the sequences and the alignment matrix
    seq1_gpu = cuda.mem_alloc(seq1.nbytes)
    seq2_gpu = cuda.mem_alloc(seq2.nbytes)
    matrix_gpu = cuda.mem_alloc(n*m*4)

    # copy sequences and matrix to the GPU
    cuda.memcpy_htod(seq1_gpu, seq1)
    cuda.memcpy_htod(seq2_gpu, seq2)
    cuda.memcpy_htod(matrix_gpu, np.zeros((n, m), dtype=np.int32))

    # compile the CUDA kernel
    diagonal_fill = mod.get_function("diagonal_fill")

    # call the kernel for each diagonal
    block_size = (32, 32, 1)
    grid=(math.ceil(a.shape[1]/32), math.ceil(a.shape[0]/32), 1))
    sum = 0
    for i in range(n+m):
      # for j in range(i+1):
      #   indexes.append(np.array([i-j,j], dtype=np.int32))
      # indexes = np.array(indexes)
      # indexes_gpu = cuda.mem_alloc(indexes.nbytes)
      # cuda.memcpy_htod(indexes_gpu, indexes)
      duration = diagonal_fill(matrix_gpu, np.int32(i), np.int32(max_diag_len), seq1_gpu, seq2_gpu, np.int32(n), np.int32(m), np.int32(gap_penalty), block=block_size, grid=grid_size, time_kernel=True)
      sum += duration

    print(sum, 'ms')
    # copy matrix back from GPU to host
    matrix = np.empty((n, m), dtype=np.int32)
    cuda.memcpy_dtoh(matrix, matrix_gpu)
    return matrix

def create_alignment_matrix(seq1, seq2, match_score=5, mismatch_score=-3, gap_score=-9):
    rows = len(seq1)
    cols = len(seq2)
    # Create the alignment matrix filled with 0
    alignment_matrix = np.zeros((rows,cols), dtype = int)

    # fill the matrix with scores
    for i in range(1, rows):
        for j in range(1, cols):
            # calculate the scores based on the surrounding cells
            to_add = 0
            if i <= cols and j < rows :
              if seq1[j] == seq2[i]:
                to_add = 5
              else:
                to_add = -3
              diagonal_score = alignment_matrix[i-1][j-1] + to_add
              up_score = alignment_matrix[i-1][j] + gap_score
              left_score = alignment_matrix[i][j-1] + gap_score
              alignment_matrix[i][j] = max(0, diagonal_score, up_score, left_score)
    return alignment_matrix

seq1 = '0CAGCCUCGCUUAG'
seq2 = '0AAUGCCAUUGCCGG'
gap_penalty = -9
print(np.array(list(seq1)))
print(np.array(list(seq2)))
matrix = SmithWaterman(np.array(list(seq1)),np.array(list(seq2)), gap_penalty)
matrix1 = create_alignment_matrix(np.array(list(seq1)),np.array(list(seq2)), gap_penalty)
# np.insert(matrix1, 0, int(seq1),axis = 0)
# np.insert(matrix1, 1, int(seq2),axis = 1)
print(matrix)
print()
print(matrix1)